<center>
<img src="https://raw.githubusercontent.com/dvgodoy/PyTorch101_ODSC_Europe2020/master/images/linear_dogs.jpg" height="400" width="800"> 

# Глубокое обучение и вообще: домашнее задание 1
</center>

## О задании

Это задание будет состоять из нескольких частей.

- __[2 балла]__ В первой части вы попробуете попрактиковаться в работе с тензорами.
- __[2.5 балла]__ Во второй части вы напишите на pytorch свою собственную логистическую регрессию. 
- __[5.5 балла]__ В третьей части вы немного поупражняетесь с нейросетками на табличных встроенных в pytorch данных.
- __[2 балла]__ Четвёртая часть будет бонусной. В ней вам будет предложено реализовать несколько различных вариаций градиентного спуска. 

Баллы даются за выполнение отдельных пунктов. Задачи в рамках одного раздела рекомендуется решать в том порядке, в котором они даны в задании. 

## Оценивание и штрафы

Каждая из задач имеет определенную «стоимость» (указана в скобках около задачи). Максимально допустимая оценка за работу — 12 баллов. За каждый день просрочки после мягкого дедлайна снимается 1 балл. После жёсткого дедлайна работы не принимаются. Даже при опозданиии на одну секунду. Сдавайте работы заранее. Мягкий дедлайн можно отодвинуть, воспользовавшись **late days policy** 
Задание выполняется самостоятельно. «Похожие» решения считаются плагиатом и все задействованные студенты (в том числе те, у кого списали) не могут получить за него больше 0 баллов. Если вы нашли решение какого-то из заданий (или его часть) в открытом источнике, необходимо указать ссылку на этот источник (скорее всего вы будете не единственным, кто это нашел, поэтому чтобы исключить подозрение в плагиате, необходима ссылка на источник).

Любой из студентов может быть вызван на защиту любого домашнего задания. В таком случае итоговая оценка студента определяется в результате защиты.

## Формат сдачи


В форму необходимо загрузить ноутбук с выполенным заданием. Сам ноутбук называйте в формате hw-01-USERNAME.ipynb, где USERNAME — ваши фамилия и имя.

## Оформление

1. Be good, drink milk and [think of Russia](https://www.youtube.com/watch?v=jyxSFfBKMxQ)
2. Обязательно фиксируйте зерно генератора случайных чисел в экспериментах. При перезапуске кода значения не должны меняться.
3. Вверху файла подпишите фамилию, имя и какой-то занимательный факт о себе.
4. Обратите внимание, что у графиков должны быть подписаны оси, заголовок графика и при необходимости обязательно наличие легенды. 

> За отсутствие названий графиков и подписей к осям могут снижаться баллы. Все картинки должны быть самодостаточны и визуально удобны для восприятия, так чтобы не нужно было смотреть ваш код или знать задание, чтобы понять что на них изображено.


Для каждого построенного графика делайте выводы. Эти выводы не должны быть поверхностными и очевидными. Не будьте мудрым королём.

<br>

<center>
<img src="https://raw.githubusercontent.com/hse-econ-data-science/andan_2023/main/hw/king.png" width="300"> 
</center>

**Пример плохого вывода:** Синенькая линия идет вверх, а красная вниз. Черненькая идет вниз, а потом вверх. 

<br>

<center>
<img src="https://raw.githubusercontent.com/hse-econ-data-science/andan_2023/main/hw/bad_lines.png" width="600"> 
</center>

In [ ]:
import math
import os
import random
import sys

import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F

from tqdm import tqdm
import sklearn

In [ ]:
torch.__version__

# 1. Простейшие операции на pytorch

В этой части вам надо будет реализовать несколько функций активации. Вы можете пока не знать, какие у них есть свойства и чем они хороши. Разговор об этом пойдёт на будущих лекциях. Достаточно просто познакомиться с ними, вбив формулы.

__[0.25 балла]__ Cоздайте два случайных тензора (двумерных, не квадратных). Умножьте их друг на друга, результат запишите в третий тензор.

In [ ]:
x = torch.rand(6, 8)
y = torch.rand(6, 8)

z = x * y 
print(z)

__[0.25 балла]__ Реализуйте [ReLU](https://en.wikipedia.org/wiki/Rectifier_(neural_networks)) используя только pytorch, примените его к тензору x (запрещено использование модулей torch.nn и его подмодулей, а также torch.relu)

In [ ]:
def relu_forward(x):
    return torch.maximum(torch.zeros_like(x), x)

assert torch.all(F.relu(x) == relu_forward(x))

__[0.25 балла]__ Сделайте тоже самое c ELU, [**Exponential Linear Units**](http://arxiv.org/abs/1511.07289)

In [ ]:
# формулу ELU взял на вики, где и ReLU
# делаю тензор из 1 и умножаю потом на константу(только по формуле она -const) 
# получаю тензор из моих -const и сравниваю через torch

def elu_forward(x):
    return torch.maximum(torch.ones_like(x) * -1e-4 , x)

assert torch.allclose(F.elu(x), elu_forward(x), 1e-4)

__[0.25 балла]__ Сделайте тоже самое c LeakyReLU, [**Leaky Rectified Linear Unit**](http://en.wikipedia.org/wiki%2FRectifier_%28neural_networks%29%23Leaky_ReLUs)

In [ ]:
# формулу  LeakyReLU взял на вики, где и ReLU
def lrelu_forward(x, alpha):
    return torch.maximum(alpha * x, x)

assert torch.all(F.leaky_relu(x, 0.01) == lrelu_forward(x, 0.01))

__[0.25 балла]__ Теперь перейдем к немного более современным функциям активаций, например Mish, она выглядит следующим образом:

$$x \cdot tanh(ln(1+e^x))$$

In [ ]:
def mish(inputs):
    return(inputs * torch.tanh(torch.log(1 + torch.exp(inputs))))

In [ ]:
assert torch.allclose(
    mish(torch.tensor([1, 1, 1], dtype=torch.float32)),
    torch.tensor([0.8651, 0.8651, 0.8651]),
    atol=1e-4,
)

In [ ]:
assert torch.allclose(
    mish(torch.tensor([0.6376, 0.4021, 0.6656, 0.3726], dtype=torch.float64)),
    torch.tensor([0.5014, 0.2908, 0.5280, 0.2663], dtype=torch.float64),
    atol=1e-4,
)

__[0.25 балла]__ Теперь реализуем swish, она выглядит следующим образом:

$$x \cdot \sigma(x)$$

In [ ]:
def swish(x):
    return x * (1/(1+torch.exp(-x)))

In [ ]:
assert torch.allclose(
    swish(torch.tensor([1, 1, 1], dtype=torch.float32)),
    torch.tensor([0.7311, 0.7311, 0.7311]),
    atol=1e-4,
)

In [ ]:
assert torch.allclose(
    swish(torch.tensor([0.6376, 0.4021, 0.6656, 0.3726], dtype=torch.float64)),
    torch.tensor([0.4171, 0.2409, 0.4396, 0.2206], dtype=torch.float64),
    atol=1e-4,
)

## __[0.5]__ Напишите на pytorch код, который решит следующую задачу оптимизации с помощью градиентного спуска:

$$
f(x) = x^2 - 2x + 5 \to \min_{x}
$$

In [ ]:
# задаем необходимые для нашего решения переменные
x = torch.tensor([2.5], requires_grad=True) 
print(x)
alpha = 0.1
epochs = 77 # можно взять и меньше для нашей задачи 

# задаем функцию, которая будет принимать на вход Х и делать манипулцию в функцию
def function(x):
    return x**2 - 2*x + 5
#########
for epoch in range(epochs):
    y = function(x) 
    #print(y) #там уже 4 устойчиво появляется, поэтому как говорил эпохи можно уменьшить, но оставлю 77 -> nice
    y.backward()
    with torch.no_grad():
        x -= alpha * x.grad 
    x.grad.zero_() #зануляем градиент, чтобы они не вхлодили в мат. операции, как было сказано у тебя в коде)
print('_____________________')    
print('Output:', x)

# 2. Логистическая регрессия

В этом задании мы соберём на PyTorch логистическую регрессию Для её обучения будем использовать игрушечные данные. 

In [ ]:
from sklearn import datasets, preprocessing

(X, y) = datasets.make_circles(n_samples=1024, shuffle=True, 
                               noise=0.2, factor=0.4, random_state=42)

ind = np.logical_or(y == 1, X[:, 1] > X[:, 0] - 0.5)
X = X[ind, :]
m = np.array([[1, 1], [-2, 1]])
X = preprocessing.scale(X)
y = y[ind]

plt.scatter(X[:, 0], X[:, 1], c=y, cmap=plt.cm.Paired, s=20)
plt.show()

__[0.25 балла]__ Рзбейте выборку на обучающую, валидационную и тестовую. Для обучающей и валидационной выборок заведите Dataset и DataLoader: 

- для обучающей: итератор должен перемешивать выборку каждую эпоху и генерировать батчи размера $32$
- для валидационной: итератор должен генировать батчи размера $32,$ данные перемешивать не надо
- про тестовую выборку пока что забудьте

In [ ]:
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader

Xx_train, Xx_test, yy_train ,yy_test = train_test_split(X , y , test_size = .3 , random_state=42 )

X_val, X_testt, y_val, y_testt = train_test_split(Xx_train, yy_train, test_size = .3 , random_state=42 ) 

print('Arrays')
print("X_train shape: {}".format(Xx_train.shape))
print("X_test shape: {}".format(Xx_test.shape))
print("y_train shape: {}".format(yy_train.shape))
print("y_test shape: {}".format(yy_test.shape))
print("X_val shape: {}".format(X_val.shape))
print("y_val shape: {}".format(y_val.shape))
print('-------------------------------------------')

#https://colab.research.google.com/github/iu5git/MPPR/blob/main/notebooks/Lab3.ipynb#scrollTo=a77Fex1TIhGE
# тут можно найти как делать в dataset and dataloader,но можно и ручками быстрее сделать самостоятельно
# без циклов и тд так как там просто переводит в тензор

X_train = torch.tensor(Xx_train, dtype=torch.float32)
y_train = torch.tensor(yy_train, dtype=torch.float32)
X_val = torch.tensor(X_val, dtype=torch.float32)
y_val = torch.tensor(y_val, dtype=torch.float32)
X_test = torch.tensor(Xx_test, dtype=torch.float32)
y_test = torch.tensor(yy_test, dtype=torch.float32)

print('Tensors')
print("X_train shape: {}".format(X_train.shape))
print("X_test shape: {}".format(X_test.shape))
print("y_train shape: {}".format(y_train.shape))
print("y_test shape: {}".format(y_test.shape))
print("X_val shape: {}".format(X_val.shape))
print("y_val shape: {}".format(y_val.shape))

# присвоение в даталоадер можно глянуть с 1 пары, по аналогии сделаю и датасет
# также можно посмотреть по ссылку как делается создание TensorDataset и сделать аналогично)
train_dataset = TensorDataset(X_train, y_train)
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)


validation_dataset =  TensorDataset(X_val, y_val)
validation_dataloader = DataLoader(validation_dataset, batch_size=32) # shuffle = False по дефолту


In [ ]:
X = torch.from_numpy(X)
y = torch.from_numpy(y)

__[0.25 балла]__ реализуйте на PyTorch функцию для поиска вероятности того, что объект относится к первому классу.


$$ \hat{y} = \langle w, x \rangle $$
$$ P( y=1 \; \big| \; x, \, w) = \dfrac{1}{1 + \exp(- \langle w, x \rangle)} = \sigma(\langle w, x \rangle)$$

Обратите внимание, что по весам мы будем искать градиенты.

In [ ]:
w = torch.tensor([1.0,1.0,1.0], requires_grad=True) 

# можно взять с swish функцию сигмоиду
def sigmoid(x): 
    return 1 / (1 + torch.exp(-x))

def probability(X):
    X = torch.cat([X, torch.ones((X.shape[0], 1))], dim=1).float()
    y = X @ w
    return sigmoid(y)


Для логистической регрессии оптимальный параметр находится минимизацией кросс-энтропии (или logloss): 

$$ L(w) =  - {1 \over n} \sum_{i=1}^n \left[ {y_i \cdot log P(y_i = 1 \, | \, x_i,w) + (1-y_i) \cdot log (1-P(y_i = 1 \, | \, x_i,w))}\right] $$


__[0.25 балла]__ реализуйте в PyTorch эту функцию потерь. Пользоваться готовыми функциями потерь вроде `CrossEntropyLoss()` нельзя. Надо вбить самостоятельно. Для значений $y$, для удобства, можно сделать One Hot Encoding.

In [ ]:
def compute_loss(X, y):
    result = - torch.mean(y * torch.log(probability(X)) + (1-y) * torch.log(1-probability(X)))
    return result

__[0.5 балла]__ Напишите функцию, которая делает один шаг обучения логистической регрессии. Для градиентного спуска используйте в качестве оптимизатора `SGD`. 

In [ ]:
optimizer = torch.optim.SGD([w], lr = 0.1)

def train_step(X, y):
    optimizer.zero_grad()
    result = compute_loss(X, y)
    result.backward()
    optimizer.step()
    return result

__[1 балл]__ Напишите цикл для обучения модели. Цикл должен делать $1000$ итераций. В случае, если ошибка на валидационной выборке начала расти и растёт в течение $20$ эпох, обучение должно останавливаться. Такая стратегия обучения называется ранней остановкой (early stopping).

In [ ]:
# задаем эпохи, оптимайзер и  индикатор для лоссов, чтобы сравнивать, когда начинает расти в течение 20 эпох
# также нужно будет задать флаг, который будет собирать в себе информацию о росте лосса
EPOCH = 10000
optimizer = torch.optim.SGD([w], lr = 0.1)
validation_loss = 9999999
flag = 0
w = torch.tensor([1.0,1.0,1.0], requires_grad=True) 


for epoch in range(EPOCH):
    loss_on_train = 0.0
    for marker,target in train_dataloader:
        train_step(X_train, y_train) 

    loss_on_val = 0.0
    with torch.no_grad():
        for marker,target in validation_dataloader:
            loss_on_val += compute_loss(marker,target) 
        loss_on_val /= len(validation_dataloader.dataset)
    
    if loss_on_val < validation_loss:
        validation_loss = loss_on_val
        flag = 0
    else:
        flag += 1
        
    if flag >= 20:
        print("Ошибка на валид выборке растет уже 20 и более раз, поэтому сделаем раннюю остановку")
        print('---------------')
        print(epoch)
        break
    

__[0.25 балла]__ Измерите качество итоговой модели на тестовой выборке. 

In [ ]:
compute_loss(X_test, y_test)

# 3. Собираем нейросеть

В этом задании мы попробуем обучить нейросеть предсказывать цены на недвижимость в Бостоне.

In [ ]:
data_url = "http://lib.stat.cmu.edu/datasets/boston"
raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)

X = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
y = raw_df.values[1::2, 2]

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=123)
X_train.shape, X_test.shape

__[0.25 балла]__ Отнормируйте данные.

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

X_train_norm = scaler.fit_transform(X_train)
X_test_norm = scaler.transform(X_test)

__[0.25 балла]__ Найдите медианную цену на обучающей выборке. Используйте её в качестве прогноза. Какое качество в терминах метрики $MAPE$ получается на тестовой выборке? Для всех дальнейших вопросах о качестве используйте эту метрику.

In [ ]:
from sklearn.metrics import mean_absolute_percentage_error
print('Медианное значение цены на обучающей выборке :',np.median(y_train))
med_price = np.median(y_train) 
y_pred = med_price.repeat(len(y_test))
print('Метрика МАРЕ :',mean_absolute_percentage_error(y_test,y_pred))

__[0.5 балла]__ Подгрузите из sklearn модель линейной регрессии и модель случайного леса. Обучите их на наших данных. Какое качество у них получилось на тестовой выборке? Какой функционал они используют для обучения?

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor

linreg = LinearRegression()
linreg.fit(X_train_norm,y_train)
y_pred_lr = linreg.predict(X_test_norm)
print('Метрика МАРЕ LinReg :',mean_absolute_percentage_error(y_test,y_pred_lr))


forest = RandomForestRegressor()
forest.fit(X_train_norm,y_train)
y_pred_for = forest.predict(X_test_norm)
print('Метрика МАРЕ Forest :',mean_absolute_percentage_error(y_test,y_pred_for))


__[0.5 балла]__ Менеджер Николай обладает гуманитарным образованием. Он спрашивает вас, насколько хорошими получились модели. Объясните ему это так, чтобы он понял.

__Ответ:__  Если говорить про локальный случай, то можно сказать, что модель случайного леса оказалась лучше, поскольку дает нам меньшую ошибку на этих данных  ≈ 15%,а у лог регрессии ≈ 17%, то есть грубо говоря разница в 2%.С помощью модели случайного леса при предсказании результата мы ошибаемся в 15 случаях из 100, например. В модели лог регрессии в 17 из 100. Много или мало? Ответ будет таким: всегда лучше стараться уменьшить ошибки в модели, поскольку это даст нам лучшие результаты предсказания("более точные").
Если выйти за рамки рассматриваемых моделей, то получившиеся модели нельзя назвать "хорошими",поскольку ошибки довольно большие и их можно уменьшить с помощью подбора параметров и необходимого препроцессинга. Наши вычисления можно назвать "сырыми" и не опираться на них, как на истину, и не рассматривать данные модели как конечный инструмент для предскзаания данных.

__[0.5 балла]__ Реализуйте линейную регрессию в виде нейронной сети. Для этого используйте один полносвязный слой без функции активации.

Обучите её, использая Adam в качестве оптимизатора и $MSE$ в качестве функции потерь. Сравните получившиеся значения коэффициентов с тем, что получилось при обучении модели средствами sklearn. Сравнение делаете с помощью какого-нибудь графика. 

In [ ]:
X_train_split, X_val, y_train_split, y_val = train_test_split(X_train_norm, y_train, 
                                                              test_size = .3 , random_state=42) 

X_train = torch.tensor(X_train_split, dtype=torch.float32)
y_train = torch.tensor(y_train_split, dtype=torch.float32)
X_val = torch.tensor(X_val, dtype=torch.float32)
y_val = torch.tensor(y_val, dtype=torch.float32)


train_dataset = TensorDataset(X_train, y_train)
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)

validation_dataset =  TensorDataset(X_val, y_val)
validation_dataloader = DataLoader(validation_dataset, batch_size=32)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from IPython.display import clear_output
from tqdm.notebook import tqdm


sns.set_style('whitegrid')
plt.rcParams.update({'font.size': 15})


def plot_losses(train_losses, test_losses, train_accuracies, test_accuracies):
    clear_output()
    fig, axs = plt.subplots(1, 2, figsize=(13, 4))
    axs[0].plot(range(1, len(train_losses) + 1), train_losses, label='train')
    axs[0].plot(range(1, len(test_losses) + 1), test_losses, label='test')
    axs[0].set_ylabel('loss')

    axs[1].plot(range(1, len(train_accuracies) + 1), train_accuracies, label='train')
    axs[1].plot(range(1, len(test_accuracies) + 1), test_accuracies, label='test')
    axs[1].set_ylabel('accuracy')

    for ax in axs:
        ax.set_xlabel('epoch')
        ax.legend()

    plt.show()

In [ ]:
class linear(nn.Module):
    def __init__(self,in_features):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(in_features = in_features, out_features= 1)
        )
        
    def forward(self,x):
        return self.model(x)

In [ ]:

device  = torch.device('cpu:0' if torch.cuda.is_available() else 'cpu')
model = linear(32,13)
optimizer_adam = torch.optim.Adam(model.parameters(), lr=0.01)
criterion_MSE = torch.nn.MSELoss()

In [ ]:
def train(model, optimizer, criterion, train_load, val_load):
    train_losses = []
    val_losses = []

    for epoch in range(1000):
        model.train()
        train_loss = 0.0
        for data, target in train_load:
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()
            train_loss += loss.item() * data.size(0)
        train_loss /= len(train_loader.dataset)
        train_losses.append(train_loss)

        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for data, target in val_load:
                output = model(data)
                loss = criterion(output, target)
                val_loss += loss.item() * data.size(0)
            val_loss /= len(val_loader.dataset)
            val_losses.append(val_loss)


    plt.plot(train_losses, label="Train Loss")
    plt.plot(val_losses, label="Val Loss")
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.legend()
    plt.show()

    return model

In [ ]:
train(model, optimizer, criterion_MSE, train_dataloader, validation_dataloader)

__[0.5 балла]__ Реализуйте две разных архитектуры нейросетей, которые принципиально отличаются друг от друга. Обучите каждую из них с двумя оптимизаторами: Adam и SGD. Визуализируйте все траектории обучения на картинке. 

Не забывайте отщипнуть в рамках этого метода от обучающей выборки кусочек для валидации. Тестовую выборку мы будем использовать только для итоговой оценки качества. 

In [ ]:
class drop(nn.Module):
    def __init__ (self, in_features, hidden_size):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(in_features, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, hidden_size//4),
            nn.ReLU(),
            nn.Linear(hidden_size//4, hidden_size//8),
            nn.ReLU,
            nn.Linear(hidden_size//8, 1))
        
    def forward(self,X):
        return self.model(X)

In [ ]:
class base(nn.Module):
    def __init__(
        self, in_features, hide_neurons):
        super(base, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(in_features, hide_neurons),
            nn.ReLU(),
            nn.Linear(hide_neurons, hide_neurons // 4),
            nn.ReLU(),
            nn.Linear(hide_neurons//4, hide_neurons//8),
            nn.ReLU(),
            nn.Linear(hide_neurons//8, 1)
            )
        
    def forward(self, X):
        return self.model(X)

In [ ]:
model_adam = base(32,13).to(device)
model_sgd = base(15,25).to(device)

In [ ]:
optimizer_adam = torch.optim.Adam(model_adam.parameters(), lr=0.01)
criterion_a = torch.nn.MSELoss()

In [ ]:
train(model_adam, optimizer_adam, criterion, train_dataloader, validation_dataloader)

In [ ]:
# your code here  (ﾉ◕ヮ◕)ﾉ

__[1 балл]__ Потратьте некоторое время на модернизацию своих архитектур. Попытайтесь побить качество, которое дал случайный лес. Опишите здесь свои эксперименты что/как/почему вы пробовали. Оставьте в виде кода реализацию лучшей модели. 

__Добавлять сюда описание своих экспериментов в виде текста - обязательно! Без этого полного балла не будет.__

In [ ]:
# your code here  ٩(⁎❛ᴗ❛⁎)۶

__[0.5 балла]__ Возьмите лучшую из получившихся у вас моделей. Обучите её на неотнормированных данных и на отнормированных данных. Визуализируйте траектории обучения обеих моделей на одной картинке.

In [ ]:
# your code here  (︺︹︺)

__[0.25 балла]__ Какое поведение в этих траекториях вы ожидали увидеть? Какое поведение вы увидели в реальности? Как думаете, почему траектории ведут себя именно так? При обучении моделей выше мы использовали $MSE$. Какой есть минус у этого функционала? 

__Ответ:__

In [ ]:
# your code here ʕ•ᴥ•ʔ

__[1 балл]__  Давайте попробуем обучить ту же самую модель, используя в качестве функций потерь другие функционалы: 

- MAE
- MSLE
- LogCosh 

Каким оказывается MAPE на тестовой выборке для всех трёх случаев?

In [ ]:
# your code here  ♪┏(・o･)┛♪

__[0.25 балла]__ Какой смысл кроется за этими функциями потерь? Как думаете, какая из них лучше всего подходит для решения нашей задачи? Объясните почему. 

В этом вам поможет [глава из книги Александра Дьяконова](https://alexanderdyakonov.files.wordpress.com/2018/10/book_08_metrics_12_blog1.pdf) и [конспект из курса МО-1 Жени Соколова](https://github.com/esokolov/ml-course-hse/blob/master/2021-fall/lecture-notes/lecture02-linregr.pdf)

__Ответ:__

> Нейронные сети не очень хорошо показывают себя на табличных данных из-за их разнородности. С такими задачами обычно хорошо справляется градиентный бустинг. Нейронные сети больше подходят для работы с картинками и текстами.

**[Бонус (0.1 балла)]**  Прикрепите фотографию того, как вы начали этот февраль. Какую самую классную эмоцию вы испытали за прошедший месяц?


# 4. Бонусный трэк: градиентные спуски

В этом разделе вы можете попробовать самостоятельно реализовать разные виды градиентных спусков для логистической регрессии. В PyTorch многие оптимизаторы уже реализованы и нам остаётся их только использовать. Тут мы реализуем все эти оптимизаторы с нуля. 

In [ ]:
from sklearn import datasets, preprocessing

(X, y) = datasets.make_circles(n_samples=1024, shuffle=True, 
                               noise=0.2, factor=0.4, random_state=42)

ind = np.logical_or(y == 1, X[:, 1] > X[:, 0] - 0.5)
X = X[ind, :]
m = np.array([[1, 1], [-2, 1]])
X = preprocessing.scale(X)
y = y[ind]

plt.scatter(X[:, 0], X[:, 1], c=y, cmap=plt.cm.Paired, s=20)
plt.show()

In [ ]:
# добавим вектор из единичек
X = np.hstack((X, np.ones(X.shape[0])[:,np.newaxis]))
X

__[0.2 балла]__ реализуйте в `numpy` функцию для поиска вероятности того, что объект относится к первому классу


$$ \hat{y} = \langle w, x \rangle $$
$$ P( y=1 \; \big| \; x, \, w) = \dfrac{1}{1 + \exp(- \langle w, x \rangle)} = \sigma(\langle w, x \rangle)$$

Обратите внимание, что веса теперь идут на вход в эту функцию. 

In [ ]:
# можно также взять функцию сигмоиду, сделать ее в нумпай и написать для перемножения массивов
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def probability(X, w):
    return sigmoid(np.dot(X,w))

In [ ]:
X = np.array([[1,2], [3,4]])
y = np.array([1, 0])
w = np.array([0.5, 0.5])

assert np.allclose(probability(X, w), np.array([0.81757448, 0.97068777]))

__[0.2 балла]__ По аналогии реализуйте логистические потери. 

$$ L(w) =  - {1 \over n} \sum_{i=1}^n \left[ {y_i \cdot log P(y_i = 1 \, | \, x_i,w) + (1-y_i) \cdot log (1-P(y_i = 1 \, | \, x_i,w))}\right] $$

In [ ]:
def compute_loss(X, y, w):
    result = -1/len(y) * np.sum((y * np.log(probability(X, w)) + (1-y) * np.log(1-probability(X, w))))
    return result

In [ ]:
assert compute_loss(X, y, w) - 1.865581 < 1e-3

__[0.4 балла]__ Мы будем обучать модель методом градиентного спуска своими руками. Для этого нам придётся вычислить градиент функции потерь, представленной выше. Возьмите листочек, ручку и в бой! 

$$ \nabla_w L = ...$$

In [ ]:
def compute_grad(X, y, w):
    # Рассчитываем количество данных
    n = y.shape[0]
    # Вычисляем вероятности класса 1 для каждого примера
    t = np.dot(X, w)
    sigm = sigmoid(t)
    # Вычисляем градиент функции потерь кросс-энтропии
    grad = np.dot(X.T, (sigm - y)) / n
    return grad

In [ ]:
assert np.allclose(compute_grad(X, y, w),np.array([1.36481889, 1.75895001]))

Функция ниже предназначена для визуализации процесса обучения. 

In [ ]:
from IPython import display

h = 0.01
x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))

def visualize(X, y, w, history):
    """С помощью магии matplolib выдаёт красоты результатов классификации"""
    Z = probability(np.c_[xx.ravel(), yy.ravel()], w)
    Z = Z.reshape(xx.shape)
    plt.figure(figsize=(8,3))
    plt.subplot(1, 2, 1)
    plt.contourf(xx, yy, Z, alpha=0.8)
    plt.scatter(X[:, 0], X[:, 1], c=y, cmap=plt.cm.Paired)
    plt.xlim(xx.min(), xx.max())
    plt.ylim(yy.min(), yy.max())
    
    plt.subplot(1, 2, 2)
    plt.plot(history)
    plt.grid()
    ymin, ymax = plt.ylim()
    plt.ylim(0, ymax)
    display.clear_output(wait=True)
    plt.show()

In [ ]:
dummy_weights = np.linspace(-1, 1, 2)
visualize(X, y, dummy_weights, [2, 0.5, .25, .1])

Пришло время обучить нашу модель. Для этого вам придётся дописать кусочки функций ниже. Обязательно попробуйте поменять гиперпараметры (размер батча и скорость обучения) и посмотреть как будет изменяться анимация. 

### [0.4  балла] Mini-batch SGD

Берём несколько рандомных наблюдений и ищем градиент по ним! 

$$ w_t = w_{t-1} - \eta \dfrac{1}{m} \sum_{j=1}^m \nabla_w L(w_t, x_{i_j}, y_{i_j}) $$



In [ ]:
np.random.seed(42)

w = np.array([0, 1.]) # веса

eta= 0.1 

epochs = 100
batch_size = 32
loss = np.zeros(n_iter)
plt.figure(figsize=(12, 5))
history = []


for i in range(epochs):
    for j in range(0, X.shape[0], batch_size):
        x_batch = X[j: j + batch_size]
        #print(f'Иксы:{x_batch}')
        #print(f'Веса:{w}')
        y_batch = y[j: j + batch_size]
        gradient = compute_grad(x_batch, y_batch, w)
        w -= eta * gradient / batch_size
    loss = compute_loss(x_batch, y_batch, w)
    history.append(loss)
visualize(X, y, w, history)
plt.clf()

In [ ]:
w = np.array([0, 1.])

### [0.4 балла] Momentum SGD

Momentum это метод, который помогает стохастическому градиентному спуску сохранять направление движения. Это осуществляется за счёт добавления в выражение дополнительного слагаемого: накопленного за предыдущие шаги градиента с весом $\alpha$. 
<br>
<br>

$$ \nu_t = \alpha \nu_{t-1} + \eta\dfrac{1}{m} \sum_{j=1}^m \nabla_w L(w_t, x_{i_j}, y_{i_j}) $$
$$ w_t = w_{t-1} - \nu_t$$


In [ ]:
np.random.seed(42)

w = np.array([0, 1.0])

eta = 0.05 
alpha = 0.2
nu = np.zeros_like(w)

history = []
epochs = 100
batch_size = 4
loss = np.zeros(n_iter)
plt.figure(figsize=(12, 5))

for i in range(epochs):
    for j in range(0, X.shape[0], batch_size):
        x_batch = X[j: j + batch_size]
        y_batch = y[j: j + batch_size]
        gradient = compute_grad(x_batch, y_batch, w)
        nu = alpha * nu + eta * gradient / batch_size
        w -= nu
    loss = compute_loss(X, y, w)
    history.append(loss)

visualize(X, y, w, history)
plt.clf()


### [0.4 балла] RMSprop

В этом блоке реализуем RMSprop. Эта вариация градиентного спуска позволяет изменять скорость обучения индивидуально для каждого параметра. 

$$ G_t^j = \alpha G_{t-1}^j + (1 - \alpha) g_{tj}^2 $$
$$ w_t^j = w_{t-1}^j - \dfrac{\eta}{\sqrt{G_t^j + \varepsilon}} g_{tj} $$

In [ ]:
np.random.seed(42)

w = np.array([0, 1.]) 

eta = 0.1 
alpha = 0.9 
g2 = np.zeros_like(w)
eps = 1e-8

history = []
epochs = 100
batch_size = 4
loss = np.zeros(n_iter)
plt.figure(figsize=(12,5))

for i in range(n_iter):
    for j in range(0, X.shape[0], batch_size):
        X_batch = X[j: j + batch_size]
        y_batch = y[j: j + batch_size]
        gradient = compute_grad(x_batch, y_batch, w)
        g2 = alpha * g2 + (1 - alpha) * gradient ** 2
        w -=  eta / np.sqrt(g2 + eps) * gradient

    loss = compute_loss(X, y, w)
    history.append(loss)


visualize(X, y, w, history)
plt.clf()

Как траектории обучения различных вариаций градиентного спуска различаются между собой? Ожидаемо ли это? Почему? Что нужно сделать, чтобы реализовать Adam? 

**Траектории обучения различных вариаций градиентного спуска  различаются в зависимости от выбранного метода оптимизации и параметров обучения. Например, первые два варианта сходятся медленнее, чем метод, который использует более сложные шаги обновления, такой как  RMSProp.
Это ожидаемо, так как разные методы оптимизации имеют разные способы адаптации шага обновления к изменению градиента. Некоторые методы могут быть более эффективными для определенных задач, в то время как другие методы могут показать лучшие результаты для других задач.**


**Чтобы реализовать Adam, необходимо выполнить следующие шаги:**
- : 1 Инициализировать переменные. Например, скорость обновления и скользящее среднее квадратов градиента.
- : 2 Вычислить градиент функции потерь по всем параметрам модели.
- : 3 Обновить скользящее среднее квадратов градиента и скорость обновления.
- : 4 Вычислить скорректированные значения скользящих средних и скорости обновления.
- : 5 Обновить параметры модели с использованием скорректированных значений.
- : 6 Повторять шаги 2-5 для каждой эпохи обучения.

**Важно отметить, что параметры Adam могут зависеть от конкретной задачи и могут потребовать настройки для достижения лучших результатов.**